In [1]:
print('runing')

runing


In [325]:
from partglot.utils.predict import get_loaded_model
import numpy as np
from src.helper.visualization import visualize_pointclouds_parts_partglot

import pymeshlab as pm
import torch
from partglot.utils.neural_utils import tokenizing
from sklearn.cluster import KMeans



part_names = ["back", "seat", "leg", "arm"]

part_semantic_groups = {
    "back": ["back"],
    "seat": ["seat"],
    "leg": ["leg", "wheel", "base"],
    "arm": ["arm"],
}


def sort_arrays(arrays):
    ref_array = arrays[0]
    sorted_indices = ref_array.argsort()
    out = []
    for a in arrays:
        out.append(a[sorted_indices])
    return out

def random_sample_array(arr: np.array, size: int = 1, with_replacement:bool=True) -> np.array:
    if with_replacement:
        while len(arr) < size:
            arr = np.concatenate([arr, arr])
    return arr[np.random.choice(len(arr), size=size, replace=False)]

def cluster_supsegs(sorted_labels, sorted_pc, sup_seg_size=512):
    sup_segs, labels = [], []
    for lbl in np.unique(sorted_labels):
        indices = np.where(sorted_labels==lbl)[0]
        tmp_pc = sorted_pc[indices]
        tmp_lbl = sorted_labels[indices]
        sup_segs.append(random_sample_array(tmp_pc, sup_seg_size))
        # labels.append(random_sample_array(tmp_lbl, sup_seg_size))
        labels.append(lbl)
    return np.array(sup_segs), np.array(labels)


def get_attn_mask_objects(pc, pc2label):
    """
    Returns ordered point cloud and mask indices in our format.
    """
    stacked_pc = np.vstack(np.vstack(pc))
    
    arg_sort = pc2label.argsort()
    
    out_pc2label, out_pg_pc = pc2label[arg_sort], np.vstack(stacked_pc)[arg_sort]

    mask = {}
    for i, pn in enumerate(part_names):
        tmp = np.where(out_pc2label == i)[0]
        print(tmp.shape)
        if tmp.shape[0] == 0:
            continue
        mask[pn] = [tmp.min(), tmp.max()]
    
    return {"mask_vertices": mask}, out_pg_pc



In [350]:
sample_idx = 1
run_gt = True
n_ssseg_custom = 10


np.random.seed(0)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


model_dir = "/home/bellatini/DL3D-Practical/models/pn_agnostic.ckpt"
# data_dir = "/home/bellatini/DL3D-Practical/data/partglot"
data_dir = "/home/bellatini/DL3D-Practical/data/partglot_100"

partglot, partglot_dm = get_loaded_model(data_dir=data_dir, model_path=model_dir)
sup_segs2label, pc2label = partglot.get_attn_maps()[sample_idx]
segs, masks = partglot_dm.h5_data['data'][sample_idx], partglot_dm.h5_data['mask'][sample_idx]

partglot.to(device)


batch_data = torch.from_numpy(partglot_dm.h5_data['data'][sample_idx:sample_idx+1]).unsqueeze(dim=1).float().to(device)
mask_data = torch.from_numpy(partglot_dm.h5_data['mask'][sample_idx:sample_idx+1]).unsqueeze(dim=1).float().to(device)

pc = np.vstack((np.vstack(np.vstack(batch_data.cpu().numpy()))))

kmeans = KMeans(n_clusters=n_ssseg_custom, random_state=1).fit(pc)
pc2sup_segs_kmeans = kmeans.labels_
np.unique(pc2sup_segs_kmeans, return_counts=True)

sorted_labels, sorted_pc = sort_arrays((kmeans.labels_, pc))

super_segs, pc2sup_segs = cluster_supsegs(sorted_labels, sorted_pc)

custom_ssegs_batch = torch.from_numpy(np.array([[super_segs]])).float().to(device)
custom_mask_batch = torch.from_numpy(np.array([[np.ones(custom_ssegs_batch.shape[2])]])).float().to(device)
custom_ssegs_batch.shape, custom_mask_batch.shape


if run_gt:
    final_ssegs_batch = batch_data
    final_mask_batch = mask_data 
    super_segs = batch_data[0][0].cpu().numpy() # gt_super_segs / super_segs
else:
    final_ssegs_batch = custom_ssegs_batch 
    final_mask_batch = custom_mask_batch 
    
attn_maps = []
for pn in part_names:
    text_embeddings = tokenizing(partglot_dm.word2int, f"chair with a {pn}").to(device)[None].expand(
        1, -1
    )
    tmp = partglot.forward(
        final_ssegs_batch, # custom_ssegs_batch / batch_data
        final_mask_batch, # custom_mask_batch / mask_data
        text_embeddings, True)
    attn_maps.append(tmp)
    

attn_maps_concat = torch.cat(attn_maps).max(0)[1].cpu().numpy()
sup_segs2label = np.squeeze(attn_maps_concat)
sup_segs2label

# CHANGE THIS TO TWEAT RUN!
K, n_points, coord = super_segs.shape

sup_segs2label_sorted=[] # pc2sup_segs: is actually pc2label
for lbl in sup_segs2label:
    tmp = np.ones(n_points) * lbl
    sup_segs2label_sorted.append(tmp)
    
sup_segs2label_sorted = np.concatenate(sup_segs2label_sorted).astype(int)

assign_ft = lambda x: sup_segs2label[x]

pc2label_sorted = assign_ft(pc2sup_segs.astype(int))

# import numpy as np
# pc2label_prefinal = []
# for lbl in sup_segs2label:
#     tmp = np.ones(512)*lbl
#     pc2label_prefinal.append(tmp)
# pc2label_prefinal = np.concatenate(pc2label_prefinal)

pc_final = np.vstack(super_segs)

# final_mask, final_pc = get_attn_mask_objects(super_segs, pc2label)
final_mask, final_pc = get_attn_mask_objects(pc_final, sup_segs2label_sorted) # pc2sup_segs: is actually pc2label

out = []
for s,f in final_mask['mask_vertices'].values():
    tmp = final_pc[s:f].astype(float)
    out.append(tmp)
    
visualize_pointclouds_parts_partglot(np.array(out))


write state dict
(19968,)
(1536,)
(3584,)
(512,)


/tmp/ipykernel_974786/1939882574.py:94: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  visualize_pointclouds_parts_partglot(np.array(out))


Output()

array([1, 0, 1, 2, 1, 1, 3, 2, 3, 1, 0, 2, 0, 0, 0, 1, 2, 2, 2, 3, 0, 3,
       1, 1, 1])

In [343]:
sup_segs2label

array([1, 0, 1, 2, 1, 1, 3, 2, 3, 1, 0, 2, 0, 0, 0, 1, 2, 2, 2, 3, 0, 3,
       1, 1, 1])

(20480,)
(2048,)
(3072,)
(0,)


({'mask_vertices': {'back': [0, 20479],
   'seat': [20480, 22527],
   'leg': [22528, 25599]}},
 array([[ 0.36761585,  0.2305407 ,  0.18729138],
        [ 0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ],
        ...,
        [-0.35350877, -0.45267278,  0.2199387 ],
        [ 0.3291822 , -0.66550934, -0.5016891 ],
        [ 0.3207059 , -0.6468924 ,  0.2737381 ]], dtype=float32))

array([0, 1, 2, 3])

In [13]:
attn_maps_concat

array([[[0, 0, 2, 3, 1, 0, 2, 1, 3, 3, 2, 1, 2, 1, 3, 2, 0, 3, 0, 0, 2,
         2, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]]])

In [124]:
np.unique(final_pc, axis=0).shape

(1729, 3)

In [91]:
np.vstack(super_segs).shape

(12800, 3)

In [92]:
pc2label.shape[0] /2

6400.0

In [110]:
final_pc.shape

(12800, 3)

In [18]:
out = []
for s,f in final_mask['mask_vertices'].values():
    tmp = final_pc[s:f]
    out.append(tmp)

In [20]:
pc2label_ref = np.load("/home/bellatini/DL3D-Practical/Baselines/PartGlot/logs/pre_trained/pn_agnostic/12-12_14-37-13/pred_label/final/0_pc_label.npy")

In [22]:
pc2label_ref.shape

(2691,)

In [108]:
pc2label_ref.shape, np.unique(pc2label_ref)

((2691,), array([0, 1, 2, 3]))

In [95]:
# torch.cat(attn_maps).shape

In [19]:
visualize_pointclouds_parts_partglot(out)

Output()

In [36]:
visualize_pointclouds_parts_partglot(segs)

Output()

In [37]:
pc2label

array([3, 3, 3, ..., 0, 0, 0])